In [1]:
import pandas as pd
import numpy as np
from datasist.structdata import detect_outliers
from sklearn.impute import SimpleImputer


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

df = pd.read_csv("train.csv", low_memory=False)
df


,ID,Customer_ID,Month,Name,Age,SSN,Occupation,Annual_Income,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,Delay_from_due_date,Num_of_Delayed_Payment,Changed_Credit_Limit,Num_Credit_Inquiries,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,0x1602,CUS_0xd40,January,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,11.27,4.0,_,809.98,26.822620,22 Years and 1 Months,No,49.574949,80.41529543900253,High_spent_Small_value_payments,312.49408867943663,Good
1,0x1603,CUS_0xd40,February,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",-1,NaN,11.27,4.0,Good,809.98,31.944960,NaN,No,49.574949,118.28022162236736,Low_spent_Large_value_payments,284.62916249607184,Good
2,0x1604,CUS_0xd40,March,Aaron Maashoh,-500,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",3,7,_,4.0,Good,809.98,28.609352,22 Years and 3 Months,No,49.574949,81.699521264648,Low_spent_Medium_value_payments,331.2098628537912,Good
3,0x1605,CUS_0xd40,April,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,NaN,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",5,4,6.27,4.0,Good,809.98,31.377862,22 Years and 4 Months,No,49.574949,199.4580743910713,Low_spent_Small_value_payments,223.45130972736786,Good
4,0x1606,CUS_0xd40,May,Aaron Maashoh,23,821-00-0265,Scientist,19114.12,1824.843333,3,4,3,4,"Auto Loan, Credit-Builder Loan, Personal Loan,...",6,NaN,11.27,4.0,Good,809.98,24.797347,22 Years and 5 Months,No,49.574949,41.420153086217326,High_spent_Medium_value_payments,341.48923103222177,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,0x25fe9,CUS_0x942c,April,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",23,7,11.5,3.0,_,502.38,34.663572,31 Years and 6 Months,No,35.104023,60.97133255718485,High_spent_Large_value_payments,479.86622816574095,Poor
99996,0x25fea,CUS_0x942c,May,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",18,7,11.5,3.0,_,502.38,40.565631,31 Years and 7 Months,No,35.104023,54.18595028760385,High_spent_Medium_value_payments,496.651610435322,Poor
99997,0x25feb,CUS_0x942c,June,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,6,5729,2,"Auto Loan, and Student Loan",27,6,11.5,3.0,Good,502.38,41.255522,31 Years and 8 Months,No,35.104023,24.02847744864441,High_spent_Large_value_payments,516.8090832742814,Poor
99998,0x25fec,CUS_0x942c,July,Nicks,25,078-73-5990,Mechanic,39628.99,3359.415833,4,6,7,2,"Auto Loan, and Student Loan",20,NaN,11.5,3.0,Good,502.38,33.638208,31 Years and 9 Months,No,35.104023,251.67258219721603,Low_spent_Large_value_payments,319.1649785257098,Standard


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        100000 non-null  object 
 1   Customer_ID               100000 non-null  object 
 2   Month                     100000 non-null  object 
 3   Name                      90015 non-null   object 
 4   Age                       100000 non-null  object 
 5   SSN                       100000 non-null  object 
 6   Occupation                100000 non-null  object 
 7   Annual_Income             100000 non-null  object 
 8   Monthly_Inhand_Salary     84998 non-null   float64
 9   Num_Bank_Accounts         100000 non-null  int64  
 10  Num_Credit_Card           100000 non-null  int64  
 11  Interest_Rate             100000 non-null  int64  
 12  Num_of_Loan               100000 non-null  object 
 13  Type_of_Loan              88592 non-null   ob

# Data Cleansing

1. **Not Useful Columns:**
   - `ID`, `Name`, `month` and `SSN` are not useful for the model. and Delete customers ID after being used

2. **Change Object Values to Float Values:**
   - `Age`, `Annual_Income`, `Num_of_Loan`, `Num_of_Delayed_Payment`, `Changed_Credit_Limit`, `Amount_invested_monthly`, `Outstanding_Debt`, `Credit_Mix`, `Monthly_Balance` are numerical but show as objects. 

3. **Values "__" in Columns:**
   - `Occupation` and `CreditMix` have values "__" that need to be addressed. And rewritten CreditMix category into float variables and Occupation into 15 different columns

4. **Outliers:**
   - Means and Delete Age rows that higher than 100 and lower than 0
   
5. **Num_Credit_Card Zeros:**
   - Num_Credit_Card that has 0

6. **Type_of_Loan Restructuring:**
   - `Type_of_Loan` needs to be rewritten as 8 columns.

7. **Negative Values in Num_Bank_Accounts:**
   - `Num_Bank_Accounts` Handle negative Values.

8. **Feature Engineering:**
   - `Credit_History_Age`, `Payment_of_Min_Amount`, `Payment_Behaviour`

9. **Missing Data:**
    - Need to fix missing Data

---

1. **Not Useful Columns:**

In [4]:
del df['ID'] 
del df['Name']
del df['SSN']
del df['Month']

2. **Numeric Columns Incorrectly Typed as Categorical:**


In [5]:
N_to_fix = ['Age', 'Annual_Income', 'Num_of_Loan', 'Num_of_Delayed_Payment'
            , 'Changed_Credit_Limit', 'Amount_invested_monthly'
            , 'Outstanding_Debt' , 'Monthly_Balance']

def fix_nums(num):
    try : 
        return float(num.replace("_",""))
    except :
        return np.nan
    
for col in N_to_fix :
    df[col] = df[col].apply(fix_nums)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 24 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Customer_ID               100000 non-null  object 
 1   Age                       100000 non-null  float64
 2   Occupation                100000 non-null  object 
 3   Annual_Income             100000 non-null  float64
 4   Monthly_Inhand_Salary     84998 non-null   float64
 5   Num_Bank_Accounts         100000 non-null  int64  
 6   Num_Credit_Card           100000 non-null  int64  
 7   Interest_Rate             100000 non-null  int64  
 8   Num_of_Loan               100000 non-null  float64
 9   Type_of_Loan              88592 non-null   object 
 10  Delay_from_due_date       100000 non-null  int64  
 11  Num_of_Delayed_Payment    92998 non-null   float64
 12  Changed_Credit_Limit      97909 non-null   float64
 13  Num_Credit_Inquiries      98035 non-null   fl

5. **Num_Credit_Card Zeros:**

In [7]:
df['Num_Credit_Card'].replace(0,1,inplace=True)

6. **Type_of_Loan Restructuring:**

In [8]:
df[['Type_of_Loan']]


,Type_of_Loan
0,"Auto Loan, Credit-Builder Loan, Personal Loan,..."
1,"Auto Loan, Credit-Builder Loan, Personal Loan,..."
2,"Auto Loan, Credit-Builder Loan, Personal Loan,..."
3,"Auto Loan, Credit-Builder Loan, Personal Loan,..."
4,"Auto Loan, Credit-Builder Loan, Personal Loan,..."
...,...
99995,"Auto Loan, and Student Loan"
99996,"Auto Loan, and Student Loan"
99997,"Auto Loan, and Student Loan"
99998,"Auto Loan, and Student Loan"


In [9]:
for i in df['Type_of_Loan'].value_counts().head(9).index[1:] : 
    df[i] = df['Type_of_Loan'].str.contains(i)

del df['Type_of_Loan']

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 31 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Customer_ID               100000 non-null  object 
 1   Age                       100000 non-null  float64
 2   Occupation                100000 non-null  object 
 3   Annual_Income             100000 non-null  float64
 4   Monthly_Inhand_Salary     84998 non-null   float64
 5   Num_Bank_Accounts         100000 non-null  int64  
 6   Num_Credit_Card           100000 non-null  int64  
 7   Interest_Rate             100000 non-null  int64  
 8   Num_of_Loan               100000 non-null  float64
 9   Delay_from_due_date       100000 non-null  int64  
 10  Num_of_Delayed_Payment    92998 non-null   float64
 11  Changed_Credit_Limit      97909 non-null   float64
 12  Num_Credit_Inquiries      98035 non-null   float64
 13  Credit_Mix                100000 non-null  ob

7. **Negative Values in Num_Bank_Accounts:**

In [11]:
df['Num_Bank_Accounts'] = df['Num_Bank_Accounts'].apply(lambda x :abs (x))

8. **Feature Engineering:**
   - `Credit_History_Age`, `Payment_of_Min_Amount`, `Payment_Behaviour`.


**Credit_History_Age**

In [12]:
print(df['Credit_History_Age'].sample(10))

11998     15 Years and 4 Months
70751    15 Years and 11 Months
95456     18 Years and 2 Months
23630                       NaN
83377      7 Years and 4 Months
46850     10 Years and 9 Months
96911    15 Years and 11 Months
75847      5 Years and 5 Months
47044     18 Years and 3 Months
88425     23 Years and 5 Months
Name: Credit_History_Age, dtype: object


In [13]:
import re

def History_age(age):
    try: 
        numbers = re.findall(r'\d+', age)
        years = int(numbers[0]) if numbers else 0
        months = int(numbers[1]) if len(numbers) > 1 else 0
        return years * 12 + months
    except:
        return np.nan

df['Credit_History_Age'] = df['Credit_History_Age'].apply(History_age)

**Payment_of_Min_Amount**

In [14]:
df['Payment_of_Min_Amount'].replace("NM","No",inplace=True)
df['Payment_of_Min_Amount'].value_counts()

Yes    52326
No     47674
Name: Payment_of_Min_Amount, dtype: int64

**Payment_Behaviour**

In [15]:
df['Payment_Behaviour']= df['Payment_Behaviour'].replace("!@9#%8",np.nan)
df['Payment_Behaviour'].value_counts()

Low_spent_Small_value_payments      25513
High_spent_Medium_value_payments    17540
Low_spent_Medium_value_payments     13861
High_spent_Large_value_payments     13721
High_spent_Small_value_payments     11340
Low_spent_Large_value_payments      10425
Name: Payment_Behaviour, dtype: int64

3. **Values "__" in Columns:**

In [16]:
df['Credit_Mix'].value_counts()

Standard    36479
Good        24337
_           20195
Bad         18989
Name: Credit_Mix, dtype: int64

In [17]:
m = {
    "Bad":0,
    "Standard":1,
    "Good":2,
    "_":np.nan
}
df['Credit_Mix'] = df['Credit_Mix'].map(m)
df['Credit_Mix'].value_counts()

1.0    36479
2.0    24337
0.0    18989
Name: Credit_Mix, dtype: int64

In [18]:
df['Occupation'].value_counts()

_______          7062
Lawyer           6575
Architect        6355
Engineer         6350
Scientist        6299
Mechanic         6291
Accountant       6271
Developer        6235
Media_Manager    6232
Teacher          6215
Entrepreneur     6174
Doctor           6087
Journalist       6085
Manager          5973
Musician         5911
Writer           5885
Name: Occupation, dtype: int64

In [19]:
customers_with_placeholder = df[df['Occupation'] == "_______"]['Customer_ID'].unique()
#mode for each customer group
mode_occupation_by_customer = df.groupby('Customer_ID')['Occupation'].apply(lambda x: x.mode()[0]).reset_index()
#replace '_______' with mode using merge
df = df.merge(mode_occupation_by_customer, on='Customer_ID', how='left', suffixes=('', '_mode'))
df['Occupation'] = df.apply(lambda row: row['Occupation_mode'] if row['Occupation'] == "_______" else row['Occupation'], axis=1)
df.drop(['Occupation_mode'], axis=1, inplace=True)


In [20]:
df['Occupation'].value_counts()

Lawyer           7096
Engineer         6864
Architect        6824
Mechanic         6776
Scientist        6744
Accountant       6744
Developer        6720
Media_Manager    6715
Teacher          6672
Entrepreneur     6648
Doctor           6568
Journalist       6536
Manager          6432
Musician         6352
Writer           6304
_______             5
Name: Occupation, dtype: int64

In [21]:
df['Occupation'] = df['Occupation'].replace("_______",df['Occupation'].mode()[0])
df['Occupation'].value_counts()


Lawyer           7101
Engineer         6864
Architect        6824
Mechanic         6776
Scientist        6744
Accountant       6744
Developer        6720
Media_Manager    6715
Teacher          6672
Entrepreneur     6648
Doctor           6568
Journalist       6536
Manager          6432
Musician         6352
Writer           6304
Name: Occupation, dtype: int64

4. **Outliers:**

In [22]:
df.dtypes

Customer_ID                  object
Age                         float64
Occupation                   object
Annual_Income               float64
Monthly_Inhand_Salary       float64
Num_Bank_Accounts             int64
Num_Credit_Card               int64
Interest_Rate                 int64
Num_of_Loan                 float64
Delay_from_due_date           int64
Num_of_Delayed_Payment      float64
Changed_Credit_Limit        float64
Num_Credit_Inquiries        float64
Credit_Mix                  float64
Outstanding_Debt            float64
Credit_Utilization_Ratio    float64
Credit_History_Age          float64
Payment_of_Min_Amount        object
Total_EMI_per_month         float64
Amount_invested_monthly     float64
Payment_Behaviour            object
Monthly_Balance             float64
Credit_Score                 object
Credit-Builder Loan          object
Personal Loan                object
Debt Consolidation Loan      object
Student Loan                 object
Payday Loan                 

In [23]:
Numericals = df.select_dtypes(exclude='object').columns[1:]

for col in Numericals:
    outliers_indices = detect_outliers(df, 0, [col])
    median = df[col].median()
    df.loc[outliers_indices, col] = median

In [24]:
df = df[(df['Age'] >= 0) & (df['Age'] <= 100)]

9. **Advanced Handling Missing Data**

In [25]:
for col in list(df.columns[-8:]):
    df[col] = df[col].astype(float)

In [26]:
df['Customer_ID'] = pd.factorize(df['Customer_ID'])[0] + 1

In [27]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=1)

In [28]:
Numericals = df.select_dtypes(exclude='object').columns[1:]
Numericals

Index(['Age', 'Annual_Income', 'Monthly_Inhand_Salary', 'Num_Bank_Accounts',
       'Num_Credit_Card', 'Interest_Rate', 'Num_of_Loan',
       'Delay_from_due_date', 'Num_of_Delayed_Payment', 'Changed_Credit_Limit',
       'Num_Credit_Inquiries', 'Credit_Mix', 'Outstanding_Debt',
       'Credit_Utilization_Ratio', 'Credit_History_Age', 'Total_EMI_per_month',
       'Amount_invested_monthly', 'Monthly_Balance', 'Credit-Builder Loan',
       'Personal Loan', 'Debt Consolidation Loan', 'Student Loan',
       'Payday Loan', 'Mortgage Loan', 'Auto Loan', 'Home Equity Loan'],
      dtype='object')

In [29]:
imputer = SimpleImputer(strategy='median')
df[Numericals[1:]] = imputer.fit_transform(df[Numericals[1:]])

In [30]:
imputer = SimpleImputer(strategy="most_frequent")
imputer.fit(df[['Payment_Behaviour']])
df[['Payment_Behaviour']] = imputer.transform(df[['Payment_Behaviour']])

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97224 entries, 0 to 99999
Data columns (total 31 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Customer_ID               97224 non-null  int64  
 1   Age                       97224 non-null  float64
 2   Occupation                97224 non-null  object 
 3   Annual_Income             97224 non-null  float64
 4   Monthly_Inhand_Salary     97224 non-null  float64
 5   Num_Bank_Accounts         97224 non-null  float64
 6   Num_Credit_Card           97224 non-null  float64
 7   Interest_Rate             97224 non-null  float64
 8   Num_of_Loan               97224 non-null  float64
 9   Delay_from_due_date       97224 non-null  float64
 10  Num_of_Delayed_Payment    97224 non-null  float64
 11  Changed_Credit_Limit      97224 non-null  float64
 12  Num_Credit_Inquiries      97224 non-null  float64
 13  Credit_Mix                97224 non-null  float64
 14  Outsta

In [32]:
m = {
    "Poor":0,
    "Standard":1,
    "Good":2
}
df['Credit_Score'] = df['Credit_Score'].map(m)
del df['Customer_ID']

In [33]:
df = pd.get_dummies(df,drop_first=False)

In [34]:
# df.to_csv('cleaned_data.csv', index=False)

In [35]:
print("Success")

Success


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 97224 entries, 0 to 99999
Data columns (total 50 columns):
 #   Column                                              Non-Null Count  Dtype  
---  ------                                              --------------  -----  
 0   Age                                                 97224 non-null  float64
 1   Annual_Income                                       97224 non-null  float64
 2   Monthly_Inhand_Salary                               97224 non-null  float64
 3   Num_Bank_Accounts                                   97224 non-null  float64
 4   Num_Credit_Card                                     97224 non-null  float64
 5   Interest_Rate                                       97224 non-null  float64
 6   Num_of_Loan                                         97224 non-null  float64
 7   Delay_from_due_date                                 97224 non-null  float64
 8   Num_of_Delayed_Payment                              97224 non-null  float64
